In [1]:
### Lanelet 2

In [2]:
import lanelet2
import lanelet2.core as lncore
import os

In [6]:
# 1. creating (programmatically) and saving a simple lane

In [11]:
# simple init an empty map
lmap = lncore.LaneletMap()

# define a lane with a given left bound and right bound (and attribs)
am = lncore.AttributeMap({"type": "lanelet"})
lb = lncore.LineString3d(lncore.getId(),
                [lncore.Point3d(lncore.getId(), 4, 0, 0), lncore.Point3d(lncore.getId(), 4, 2, 0),
                 lncore.Point3d(lncore.getId(), 3, 3, 1), lncore.Point3d(lncore.getId(), 3, 4, 1)])
rb = lncore.LineString3d(lncore.getId(),
                [lncore.Point3d(lncore.getId(), 6, 0, 0), lncore.Point3d(lncore.getId(), 6, 2, 0),
                 lncore.Point3d(lncore.getId(), 5, 3, 1), lncore.Point3d(lncore.getId(), 5, 4, 1)])
llet1 = lncore.Lanelet(lncore.getId(), lb, rb, am)

lmap.add(llet1)

# create a projector in London
projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(51.40, -0.04, 0.0))
ll_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/llet_generated_map.osm")
lanelet2.io.write(ll_path, lmap, projector)

In [12]:
# 2. examing the result

In [13]:
print("* lanelet left bound")
for p in llet1.leftBound:
    print("    %s " % p)
print("* lanelet left bound")
for p in llet1.rightBound:
    print("    %s " % p)

* lanelet left bound
    [id: 6451325976 x: 4 y: 0 z: 0] 
    [id: 6451325977 x: 4 y: 2 z: 0] 
    [id: 6451325978 x: 3 y: 3 z: 1] 
    [id: 6451325979 x: 3 y: 4 z: 1] 
* lanelet left bound
    [id: 6451325981 x: 6 y: 0 z: 0] 
    [id: 6451325982 x: 6 y: 2 z: 0] 
    [id: 6451325983 x: 5 y: 3 z: 1] 
    [id: 6451325984 x: 5 y: 4 z: 1] 


In [31]:
# 3. reading an existing OSM map
osm_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/mapping_example.osm")
print("using OSM: %s (exists? %s)" % (osm_path, os.path.exists(osm_path)))
projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(49.00,8.42))
lmap, err_list = lanelet2.io.loadRobust(osm_path, projector) 
print("%d errors, %d lanes detected" % (len(err_list), len([l for l in lmap.laneletLayer])))

using OSM: /home/developer/workspace/data/mapping_example.osm (exists? True)
0 errors, 371 lanes detected


In [32]:
# what's in a lanelet map? 
for s in dir(lmap):
    if "__" not in s:
        print s

add
areaLayer
laneletLayer
lineStringLayer
pointLayer
polygonLayer
regulatoryElementLayer


In [45]:
# let's look at the the lanelet layer 
lanes = [lane for lane in lmap.laneletLayer]
print("--- example of a lanelet ---\n\n%s" % lanes[0])
lane0 = lanes[0]
for s in dir(lane0):
    if "__" not in s:
        print "    * %s" % s

--- example of a lanelet ---

[id: 9178926741377113721, left id: 997750960516616982, right id: 2216115017118688772]
    * addRegulatoryElement
    * attributes
    * centerline
    * id
    * invert
    * inverted
    * leftBound
    * polygon2d
    * polygon3d
    * regulatoryElements
    * removeRegulatoryElement
    * rightBound
    * rightOfWay
    * speedLimits
    * trafficLights
    * trafficSigns


In [44]:
nmax = 500
print("* example of the laneletLayer (first %d)\n" % nmax)
for ln in lmap.laneletLayer:
    print ln
    nmax -= 1
    if nmax == 0:
        break

* example of the laneletLayer (first 500)

[id: 9178926741377113721, left id: 997750960516616982, right id: 2216115017118688772]
[id: 9037740909199276460, left id: 6937946819898808252, right id: 1729046099968656320]
[id: 8717970484406193818, left id: 5537827893167917386 (inverted), right id: 3128557769764614810]
[id: 8601933696747810962, left id: 2757326693420322496, right id: 2764641530780420290 (inverted)]
[id: 8396043010843852718, left id: 4860616454757881010 (inverted), right id: 2785499205350414700 (inverted)]
[id: 7599071200282796306, left id: 4406727741069666934, right id: 43190 (inverted)]
[id: 7326074532659563937, left id: 5214965862031460312, right id: 43270 (inverted)]
[id: 7195674799508775743, left id: 6960048458279195872, right id: 43178 (inverted)]
[id: 6980464299688733498, left id: 5375602761775609010, right id: 3440717602992035100 (inverted)]
[id: 6923355182620813640, left id: 5552362054548145838, right id: 2420601589665656822]
[id: 6722104362058561355, left id: 43274 (

In [ ]:
# 4. Now, open that map in JOSM
#    click on "imagery" -> "bing aerial view" to add a visual context
#    select two lanes and compute the plan

In [66]:
# 5. Routing: go from a lanelet to another

In [50]:
startLane = lmap.laneletLayer[?]
endLane = lmap.laneletLayer[?]
# ... and traffic rules (Germany is the sole location, 
#                        and luckily for us, we are in Germany)
trafficRules = lanelet2.traffic_rules.create(lanelet2.traffic_rules.Locations.Germany, 
                                             lanelet2.traffic_rules.Participants.Vehicle)
graph = lanelet2.routing.RoutingGraph(lmap, trafficRules)
sp = graph.shortestPath(startLane, endLane)
print sp

None


In [48]:
rt = graph.getRoute(startLane, endLane)
print rt

None
